In [82]:
import pandas as pd
import numpy as np
import pickle

In [83]:
df = pd.read_excel('../data/restart.xlsx').loc[0:83]
df

,Date (UTC time zone),Away Team,Home Team,Away Season PPG/OPPG,Home Season PPG/OPPG,Pre-game over/under,Pre-game spread(home),Q1 Away Score,Q1 Home Score,Q1 Projected Score,...,Q3 Away Score,Q3 Home Score,Q3 Projected Score,Final Away,Final Home,Overtimes,Q1 Prediction,Q2 Prediction,Q3 Prediction,Final Total
0,2020-07-31 01:00:00+00:00,LOS_ANGELES_CLIPPERS,LOS_ANGELES_LAKERS,116/109.6,114.2/106.8,219.0,-4.0,23.0,35.0,220.0,...,77.0,76.0,208.5,101,103.0,0.0,227.577661,218.198553,208.300,204
1,2020-07-31 18:30:00+00:00,ORLANDO_MAGIC,BROOKLYN_NETS,106.7/107.5,110.9/111.7,211.0,8.0,36.0,39.0,233.5,...,111.0,82.0,246.5,128,118.0,0.0,241.477276,238.674110,246.890,246
2,2020-07-31 20:00:00+00:00,MEMPHIS_GRIZZLIES,PORTLAND_TRAIL_BLAZERS,113/114.1,114/115.5,224.5,-4.0,30.0,35.0,235.5,...,96.0,93.0,247.5,135,140.0,1.0,240.108958,243.797056,247.630,275
3,2020-07-31 20:00:00+00:00,PHOENIX_SUNS,WASHINGTON_WIZARDS,112.7/113.9,115.6/119.7,224.5,7.5,38.0,31.0,230.5,...,89.0,81.0,225.5,125,112.0,0.0,247.118431,236.267580,228.200,237
4,2020-07-31 22:30:00+00:00,BOSTON_CELTICS,MILWAUKEE_BUCKS,113/107,118.7/107.4,220.5,-3.5,25.0,33.0,232.5,...,87.0,87.0,230.5,112,119.0,0.0,227.709058,234.846761,230.000,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2020-08-13 04:00:00+00:00,PORTLAND_TRAIL_BLAZERS,BROOKLYN_NETS,114/115.5,110.9/111.7,238.5,10.5,31.0,31.0,243.5,...,97.0,104.0,261.5,134,133.0,0.0,235.040000,255.010000,259.280,267
80,2020-08-14 04:00:00+00:00,PHILADELPHIA_76ERS,HOUSTON_ROCKETS,109.6/107.4,118.7/114.9,231.5,-2.5,39.0,26.0,237.5,...,103.0,77.0,235.5,134,96.0,0.0,237.510000,231.950000,234.980,230
81,2020-08-14 04:00:00+00:00,MIAMI_HEAT,INDIANA_PACERS,112.2/108.9,109.3/107.4,221.0,1.5,13.0,28.0,206.5,...,70.0,84.0,206.5,92,107.0,0.0,205.160000,211.496000,207.640,199
82,2020-08-14 04:00:00+00:00,OKLAHOMA_CITY_THUNDER,LOS_ANGELES_CLIPPERS,110.8/108.3,116/109.6,225.5,-3.5,22.0,21.0,212.5,...,63.0,70.0,190.5,103,107.0,1.0,209.780000,195.005000,187.014,210


In [86]:
"""
ppg = {}
oppg = {}
with open('../data/ppg_avgs.csv', 'r', newline='') as csvfile:
        rows = csvfile.readlines()
        for row in rows:
            spl = row.split(',')
            teamname = spl[0]
            ppg[teamname] = float(spl[1])
            oppg[teamname] = float(spl[2])
"""

quarter=3
with open('../Models/predict_{}.sav'.format(quarter), 'rb') as pickle_file:
    regression_model_2 = pickle.load(pickle_file)

for index in df.index:
    avg_scored = (float(df.loc[index, 'Away Season PPG/OPPG'].split('/')[0]) + float(df.loc[index, 'Home Season PPG/OPPG'].split('/')[0]))/2
    avg_allowed = (float(df.loc[index, 'Away Season PPG/OPPG'].split('/')[-1]) + float(df.loc[index, 'Home Season PPG/OPPG'].split('/')[-1]))/2

    away_score = df.loc[index, 'Q{} Away Score'.format(quarter)]
    home_score = df.loc[index, 'Q{} Home Score'.format(quarter)]

    total_score = home_score + away_score
    diff = abs(home_score-away_score)
    ans = regression_model_2.predict([[diff, total_score, avg_scored, avg_allowed]])
    df.loc[index, 'Q{} Prediction'.format(quarter)] = ans[0]
df


,Date (UTC time zone),Away Team,Home Team,Away Season PPG/OPPG,Home Season PPG/OPPG,Pre-game over/under,Pre-game spread(home),Q1 Away Score,Q1 Home Score,Q1 Projected Score,...,Q3 Away Score,Q3 Home Score,Q3 Projected Score,Final Away,Final Home,Overtimes,Q1 Prediction,Q2 Prediction,Q3 Prediction,Final Total
0,2020-07-31 01:00:00+00:00,LOS_ANGELES_CLIPPERS,LOS_ANGELES_LAKERS,116/109.6,114.2/106.8,219.0,-4.0,23.0,35.0,220.0,...,77.0,76.0,208.5,101,103.0,0.0,226.790626,217.061386,208.566189,204
1,2020-07-31 18:30:00+00:00,ORLANDO_MAGIC,BROOKLYN_NETS,106.7/107.5,110.9/111.7,211.0,8.0,36.0,39.0,233.5,...,111.0,82.0,246.5,128,118.0,0.0,242.165790,238.775985,246.729445,246
2,2020-07-31 20:00:00+00:00,MEMPHIS_GRIZZLIES,PORTLAND_TRAIL_BLAZERS,113/114.1,114/115.5,224.5,-4.0,30.0,35.0,235.5,...,96.0,93.0,247.5,135,140.0,1.0,237.802792,242.087764,246.842008,275
3,2020-07-31 20:00:00+00:00,PHOENIX_SUNS,WASHINGTON_WIZARDS,112.7/113.9,115.6/119.7,224.5,7.5,38.0,31.0,230.5,...,89.0,81.0,225.5,125,112.0,0.0,244.349165,233.518554,227.274790,237
4,2020-07-31 22:30:00+00:00,BOSTON_CELTICS,MILWAUKEE_BUCKS,113/107,118.7/107.4,220.5,-3.5,25.0,33.0,232.5,...,87.0,87.0,230.5,112,119.0,0.0,226.816998,234.231157,230.258355,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2020-08-13 04:00:00+00:00,PORTLAND_TRAIL_BLAZERS,BROOKLYN_NETS,114/115.5,110.9/111.7,238.5,10.5,31.0,31.0,243.5,...,97.0,104.0,261.5,134,133.0,0.0,232.588731,253.936124,258.459196,267
80,2020-08-14 04:00:00+00:00,PHILADELPHIA_76ERS,HOUSTON_ROCKETS,109.6/107.4,118.7/114.9,231.5,-2.5,39.0,26.0,237.5,...,103.0,77.0,235.5,134,96.0,0.0,236.065945,230.439359,234.690467,230
81,2020-08-14 04:00:00+00:00,MIAMI_HEAT,INDIANA_PACERS,112.2/108.9,109.3/107.4,221.0,1.5,13.0,28.0,206.5,...,70.0,84.0,206.5,92,107.0,0.0,202.916835,210.419160,207.752881,199
82,2020-08-14 04:00:00+00:00,OKLAHOMA_CITY_THUNDER,LOS_ANGELES_CLIPPERS,110.8/108.3,116/109.6,225.5,-3.5,22.0,21.0,212.5,...,63.0,70.0,190.5,103,107.0,1.0,208.105238,192.870324,187.251435,210


In [87]:
df['Q1_prediction_diff'] = df['Q1 Prediction'] - df['Q1 Projected Score']
df['Q1_line_diff'] = df['Final Total'] - df['Q1 Projected Score']
df['Q1_success'] = ((df['Q1_prediction_diff'] > 0) & (df['Q1_line_diff'] > 0)) | ((df['Q1_prediction_diff'] < 0) & (df['Q1_line_diff'] < 0))
df['Q2_prediction_diff'] = df['Q2 Prediction'] - df['Q2 Projected Score']
df['Q2_line_diff'] = df['Final Total'] - df['Q2 Projected Score']
df['Q2_success'] = ((df['Q2_prediction_diff'] > 0) & (df['Q2_line_diff'] > 0)) | ((df['Q2_prediction_diff'] < 0) & (df['Q2_line_diff'] < 0))
df['Q3_prediction_diff'] = df['Q3 Prediction'] - df['Q3 Projected Score']
df['Q3_line_diff'] = df['Final Total'] - df['Q3 Projected Score']
df['Q3_success'] = ((df['Q3_prediction_diff'] > 0) & (df['Q3_line_diff'] > 0)) | ((df['Q3_prediction_diff'] < 0) & (df['Q3_line_diff'] < 0))
df

,Date (UTC time zone),Away Team,Home Team,Away Season PPG/OPPG,Home Season PPG/OPPG,Pre-game over/under,Pre-game spread(home),Q1 Away Score,Q1 Home Score,Q1 Projected Score,...,Final Total,Q1_prediction_diff,Q1_line_diff,Q1_success,Q2_prediction_diff,Q2_line_diff,Q2_success,Q3_prediction_diff,Q3_line_diff,Q3_success
0,2020-07-31 01:00:00+00:00,LOS_ANGELES_CLIPPERS,LOS_ANGELES_LAKERS,116/109.6,114.2/106.8,219.0,-4.0,23.0,35.0,220.0,...,204,6.790626,-16.0,False,1.561386,-11.5,False,0.066189,-4.5,False
1,2020-07-31 18:30:00+00:00,ORLANDO_MAGIC,BROOKLYN_NETS,106.7/107.5,110.9/111.7,211.0,8.0,36.0,39.0,233.5,...,246,8.665790,12.5,True,5.275985,12.5,True,0.229445,-0.5,False
2,2020-07-31 20:00:00+00:00,MEMPHIS_GRIZZLIES,PORTLAND_TRAIL_BLAZERS,113/114.1,114/115.5,224.5,-4.0,30.0,35.0,235.5,...,275,2.302792,39.5,True,0.587764,33.5,True,-0.657992,27.5,False
3,2020-07-31 20:00:00+00:00,PHOENIX_SUNS,WASHINGTON_WIZARDS,112.7/113.9,115.6/119.7,224.5,7.5,38.0,31.0,230.5,...,237,13.849165,6.5,True,-1.981446,1.5,False,1.774790,11.5,True
4,2020-07-31 22:30:00+00:00,BOSTON_CELTICS,MILWAUKEE_BUCKS,113/107,118.7/107.4,220.5,-3.5,25.0,33.0,232.5,...,231,-5.683002,-1.5,True,2.731157,-0.5,False,-0.241645,0.5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2020-08-13 04:00:00+00:00,PORTLAND_TRAIL_BLAZERS,BROOKLYN_NETS,114/115.5,110.9/111.7,238.5,10.5,31.0,31.0,243.5,...,267,-10.911269,23.5,False,-3.563876,9.5,False,-3.040804,5.5,False
80,2020-08-14 04:00:00+00:00,PHILADELPHIA_76ERS,HOUSTON_ROCKETS,109.6/107.4,118.7/114.9,231.5,-2.5,39.0,26.0,237.5,...,230,-1.434055,-7.5,True,-1.060641,-1.5,True,-0.809533,-5.5,True
81,2020-08-14 04:00:00+00:00,MIAMI_HEAT,INDIANA_PACERS,112.2/108.9,109.3/107.4,221.0,1.5,13.0,28.0,206.5,...,199,-3.583165,-7.5,True,-1.080840,-12.5,True,1.252881,-7.5,False
82,2020-08-14 04:00:00+00:00,OKLAHOMA_CITY_THUNDER,LOS_ANGELES_CLIPPERS,110.8/108.3,116/109.6,225.5,-3.5,22.0,21.0,212.5,...,210,-4.394762,-2.5,True,-3.629676,13.5,False,-3.248565,19.5,False


In [88]:
def get_continuous_success(q):
    for num in range(0, int(max(df['Q{}_prediction_diff'.format(q)]))):
        values = df[abs(df['Q{}_prediction_diff'.format(q)]) > num]['Q{}_success'.format(q)].value_counts()
        num_instances = len(df[abs(df['Q{}_prediction_diff'.format(q)]) > num])
        try:
            values[False]
        except:
            print("100% of successes at {} confidence level".format(num) + "(on " + str(num_instances) + " instances)")
            continue
        print(str("%.2f" % ((values[True])/(values[True] + values[False])*100)) + "% of successes at {} confidence level".format(num)+ "(on " + str(num_instances) + " instances)")

In [89]:
get_continuous_success(1)

58.33% of successes at 0 confidence level(on 84 instances)
59.15% of successes at 1 confidence level(on 71 instances)
63.64% of successes at 2 confidence level(on 55 instances)
67.39% of successes at 3 confidence level(on 46 instances)
63.16% of successes at 4 confidence level(on 38 instances)
64.29% of successes at 5 confidence level(on 28 instances)
66.67% of successes at 6 confidence level(on 24 instances)
76.47% of successes at 7 confidence level(on 17 instances)
91.67% of successes at 8 confidence level(on 12 instances)
85.71% of successes at 9 confidence level(on 7 instances)
80.00% of successes at 10 confidence level(on 5 instances)
100% of successes at 11 confidence level(on 3 instances)
100% of successes at 12 confidence level(on 2 instances)


In [90]:
get_continuous_success(2)

53.57% of successes at 0 confidence level(on 84 instances)
52.24% of successes at 1 confidence level(on 67 instances)
56.60% of successes at 2 confidence level(on 53 instances)
61.11% of successes at 3 confidence level(on 36 instances)
63.16% of successes at 4 confidence level(on 19 instances)
61.54% of successes at 5 confidence level(on 13 instances)
50.00% of successes at 6 confidence level(on 10 instances)
42.86% of successes at 7 confidence level(on 7 instances)
60.00% of successes at 8 confidence level(on 5 instances)
66.67% of successes at 9 confidence level(on 3 instances)
100% of successes at 10 confidence level(on 2 instances)
100% of successes at 11 confidence level(on 1 instances)


In [91]:
get_continuous_success(3)

52.38% of successes at 0 confidence level(on 84 instances)
51.61% of successes at 1 confidence level(on 62 instances)
55.81% of successes at 2 confidence level(on 43 instances)
62.07% of successes at 3 confidence level(on 29 instances)
68.42% of successes at 4 confidence level(on 19 instances)
71.43% of successes at 5 confidence level(on 14 instances)
75.00% of successes at 6 confidence level(on 8 instances)
80.00% of successes at 7 confidence level(on 5 instances)
